# Go-Explore Red Teaming (Dual) — Starter Notebook\nThis notebook shows how to create a `submission.zip` with `attack.py` and `guardrail.py`, and run the local evaluator.\n

In [ ]:
import shutil, zipfile, os\nfrom pathlib import Path\n\nROOT = Path('.')\nSUBDIR = ROOT/'submission'\nSUBDIR.mkdir(exist_ok=True)\n\n# Copy baselines as your starting point\nshutil.copy('aicomp_sdk/baselines/attacker_goexplore.py', SUBDIR/'attack.py')\nshutil.copy('aicomp_sdk/baselines/guardrail_rules.py', SUBDIR/'guardrail.py')\n\n# Build submission.zip\nzip_path = ROOT/'submission.zip'\nwith zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as z:\n    z.write(SUBDIR/'attack.py', arcname='attack.py')\n    z.write(SUBDIR/'guardrail.py', arcname='guardrail.py')\nzip_path\n

In [ ]:
!python evaluation.py --submission_zip submission.zip --seconds 30\n

## Next steps\n- Edit `submission/attack.py` and `submission/guardrail.py`\n- Rebuild `submission.zip`\n- Submit the notebook to Kaggle (or upload `submission.zip` depending on your competition config).\n